In [1]:
import os

In [2]:
os.chdir("../")
%pwd

'c:\\Users\\2101\\OneDrive\\Desktop\\git\\Brain-Tumor-MRI-Classification'

In [3]:
import tensorflow as tf

In [4]:
model=tf.keras.models.load_model("artifacts/training/brain_model.h5")

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params : dict
    params_image_size : list
    params_batch_size:int

In [6]:
from brainTumorMRIClassification.constants import *
from brainTumorMRIClassification.utils.common import read_yaml, create_directories, save_json

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH,params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_validation_config(self)->EvaluationConfig:
        return EvaluationConfig(
            path_of_model=Path("artifacts/training/brain_model.h5"),
            training_data=Path("artifacts/data_ingestion/brain-mri/Training"),
            all_params = self.params,
            params_image_size = self.params.IMAGE_SIZE,
            params_batch_size = self.params.BATCH_SIZE
        )

In [8]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    def _valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1. / 255,
            validation_split=0.30
        )
        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:2],
            interpolation='bilinear'
        )
        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )
        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs
        )

    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = self.model.evaluate(self.valid_generator)

    def save_score(self):
        scores = {
            "loss": self.score[0],
            "accuracy": self.score[1]
        }
        save_json(path=Path("scores.json"), data=scores)


In [9]:
try:
    config=ConfigurationManager()
    val_config=config.get_validation_config()
    evaluation = Evaluation(config=val_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e

Found 1712 images belonging to 4 classes.


c:\Users\2101\OneDrive\Desktop\git\Brain-Tumor-MRI-Classification\venv\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


54/54 ━━━━━━━━━━━━━━━━━━━━ 22s 373ms/step - accuracy: 0.9918 - loss: 0.0250
